# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-10 15:01:10,942 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedMunicio','compliance_municipio')

2024-10-10 15:01:15,668 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedMunicio)
2024-10-10 15:01:27,213 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_municipio
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-10 15:01:27,229 - INFO - Aguarde... Executando Query (Delta)
2024-10-10 15:01:27,230 - INFO - 
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_municipio

2024-10-10 15:01:27,353 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedMunicio'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:02:34,671 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:02:43,016 - INFO - Dados persistidos com sucesso
2024-09-25 19:02:43,017 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedMunicio
2024-09-25 19:02:43,018 - INFO - Aguarde... Realizando optimize
2024-09-25 19:02:46,866 - INFO - Optimize executado com sucesso.
2024-09-25 19:02:46,867 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:03:09,795 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [7]:
extrator_bronze.parar_sessao()

2024-10-10 15:01:49,901 - INFO - Sessão Spark finalizada.
